# H5XRAY

Examples using h5xray visualizing HDF5 files stored in S3.

(IN PROGRESS)

__Jonathan Markel__  
3D Geospatial Laboratory  
The University of Texas at Austin  
11/06/2023

#### [Twitter](https://twitter.com/jonm3d) | [GitHub](https://github.com/jonm3d) | [Website](http://j3d.space) | [GoogleScholar](https://scholar.google.com/citations?user=KwxwFgYAAAAJ&hl=en) | [LinkedIn](https://www.linkedin.com/in/j-markel/) 

In [ ]:
from h5xray import h5xray

## ICESat-2 Data from S3
Let's use some sample ICESat-2 H5 files already on S3 (several GB of geolocated photon data). We'll be combining several resources to locate some ICESat-2 data in the cloud
- NASA OpenScapes [Data Access Using S3](https://nasa-openscapes.github.io/earthdata-cloud-cookbook/examples/NSIDC/ICESat2-CMR-OnPrem-vs-Cloud.html#data-access-using-aws-s3) Guide 
- [ICESat-2 Cloud Data Access](https://github.com/icesat2py/icepyx/blob/main/doc/source/example_notebooks/IS2_cloud_data_access.ipynb) with icepyx

In [ ]:
# Ensure we are working on AWS US-WEST-2
# I'm using CryoCloud for notebook development...
# If you're running locally, feel free to use a sample file from /data/atl03_X.h5
h5xray.check_if_aws(verbose=True);

Commented code in the following cell was used to determine the S3 URL of a given ICESat-2 file of interest.

In [ ]:
import icepyx as ipx
import h5py
import s3fs

In [ ]:
# # Earthdata Credentials
earthdata_uid = 'your_username'
earthdata_pwd = 'your_password'

# # Create an icepyx Query Object (to make use of its login capability)
short_name = 'ATL03'
spatial_extent = [-45, 58, -35, 75]  
date_range = ['2019-11-30', '2019-11-30']  

reg = ipx.Query(short_name, spatial_extent, date_range)
reg.earthdata_login(earthdata_uid, earthdata_pwd, s3token=True)

# # Set up S3 Filesystem
s3 = s3fs.S3FileSystem(
    key=reg._s3login_credentials['accessKeyId'],
    secret=reg._s3login_credentials['secretAccessKey'],
    token=reg._s3login_credentials['sessionToken']
)

# # Specify the S3 URL
s3url = 's3://nsidc-cumulus-prod-protected/ATLAS/ATL03/006/2019/11/30/ATL03_20191130112041_09860505_006_01.h5'

In [ ]:
with s3.open(s3url, 'rb') as s3f:
    with h5py.File(s3f, 'r') as f:
        print([key for key in f.keys()])


This may take a little while to run depending on the file size and your system...

In [ ]:
h5xray.analyze(s3url, request_byte_size=4*1024**2, plotting_options=
               {'debug':True, # whether to include the title, colormap, and labels
                    'cmap': plt.cm.RdYlBu_r, 
                    'byte_threshold':10 * 1024**2, # datasets with more than this get labeled
                    'font_size':9, # font size for dataset labels
                    'figsize':(10, 3),
                    'max_requests': 15, # specify colormap range
                    'title':'S3 Access',
                   }) # default usage